In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%load_ext line_profiler

import tensorflow as tf
slim = tf.contrib.slim
tf.logging.set_verbosity(tf.logging.INFO)
sess_config = tf.ConfigProto()
#config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction=0.45

import sys
import os

ADE20K_DATA = '/gpfs01/bethge/data/ADE20K_2016_07_26'
MASK_RCNN_MODEL_PATH = 'Mask_RCNN/' 
SLIM_MODELS_PATH = 'slim/'
TRANSFORMER_MODELS_PATH = 'transformer/'
SIAMESE_MASK_RCNN_PATH = '/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/siamese-mask-rcnn/'

if MASK_RCNN_MODEL_PATH not in sys.path:
    sys.path.append(MASK_RCNN_MODEL_PATH)
if SIAMESE_MASK_RCNN_PATH not in sys.path:
    sys.path.append(SIAMESE_MASK_RCNN_PATH)
    
from samples.ade20k import ade20k
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize
    
if SLIM_MODELS_PATH not in sys.path:
    sys.path.append(SLIM_MODELS_PATH)
if TRANSFORMER_MODELS_PATH not in sys.path:
    sys.path.append(TRANSFORMER_MODELS_PATH)
    
import utils as siamese_utils
import model as siamese_model
import siamese_mrcnn_models as model_zoo
    
import time
import random
import numpy as np
import skimage.io
import imgaug
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12.0, 6.0)

from spatial_transformer import transformer

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Set COCO directory    
COCO_DIR = '/gpfs01/bethge/share/mscoco/COCO'

Instructions for updating:
Use the retry module or similar alternatives.


Using TensorFlow backend.


In [2]:
nopascal_classes = [8,10,11,12,13,14,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,60,62,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80]

In [3]:
pascal_classes = np.array(range(1,81))[np.array([i not in nopascal_classes for i in range(1,81)])]

In [4]:
class TrainConfig(ade20k.ADE20KConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    LEARNING_RATE = 0.001
    NUM_CLASSES = 2603 # 3148 + 1
    TARGET_MAX_DIM = 96
    TARGET_MIN_DIM = 75
    IMAGE_MIN_DIM = 400
    IMAGE_MAX_DIM = 512
    #IMAGE_RESIZE_MODE = 'none'
    TARGET_SHAPE = np.array([TARGET_MAX_DIM, TARGET_MAX_DIM, 3])
    TARGET_PADDING = True
    MAX_TARGET_INSTANCES = 10
    # Reduce model size for prototyping
    BACKBONE = 'resnet50'
    FPN_FEATUREMAPS = 128
    RPN_ANCHOR_STRIDE = 2
    RPN_TRAIN_ANCHORS_PER_IMAGE = 64
    POST_NMS_ROIS_TRAINING = 250
    POST_NMS_ROIS_INFERENCE = 250
    TRAIN_ROIS_PER_IMAGE = 50
    DETECTION_MAX_INSTANCES = 15
    DETECTION_NMS_THRESHOLD = 0.5
    MAX_GT_INSTANCES = 15
    LOSS_WEIGHTS = {'rpn_class_loss': 2.0, 
                    'rpn_bbox_loss': 0.2, 
                    'mrcnn_class_loss': 2.0, 
                    'mrcnn_bbox_loss': 0.5, 
                    'mrcnn_mask_loss': 1.0}
    # ACTIVE_CLASSES = np.array(range(1,81))
    ACTIVE_CLASSES =  [1831, 2855, 2978, 3050]
    
config = TrainConfig()
config.display()


Configurations:
ACTIVE_CLASSES                 [1831, 2855, 2978, 3050]
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        15
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.5
FPN_CLASSIF_FC_LAYERS_SIZE     1024
FPN_FEATUREMAPS                128
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                2615
IMAGE_MIN_DIM                  400
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 2.0, 'rpn_bbox_loss': 0.2, 'mrcnn_class_loss': 2.0, 'mrcnn_bbox_loss': 0.5, 'mrcnn_mask_loss': 1.0}

In [5]:
# Load ADE20K/train dataset
train_dataset = siamese_utils.IndexedADE20KDataset()
train_dataset.load_ade20k(ADE20K_DATA, "train")
train_dataset.prepare()
train_dataset.build_indices()

# Load ADE20K/val dataset
val_dataset = siamese_utils.IndexedADE20KDataset()
val_dataset.load_ade20k(ADE20K_DATA, "val")
val_dataset.prepare()
val_dataset.build_indices()

### New

In [6]:
# Create model object in inference mode.
model = model_zoo.NewSiameseMaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

In [7]:
# Load weights trained on Imagenet
model.load_weights('/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/logs/imagenet20180511T1119/mask_rcnn_imagenet_0810.h5', by_name=True)
model.set_log_dir()
# model.set_trainable("(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)")
# model.load_weights('/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/logs/coco20180529T1830/mask_rcnn_coco_0240.h5', by_name=True)


Re-starting from epoch 810


In [14]:
model.train(train_dataset, val_dataset, learning_rate=config.LEARNING_RATE, epochs=120, layers="heads")
model.config.LOSS_WEIGHTS = {'rpn_class_loss': 2.0/2, 'rpn_bbox_loss': 0.2/2, 'mrcnn_class_loss': 2.0, 'mrcnn_bbox_loss': 0.5, 'mrcnn_mask_loss': 1.0}
model.train(train_dataset, val_dataset, learning_rate=config.LEARNING_RATE, epochs=240, layers="4+")
# model.config.LOSS_WEIGHTS = {'rpn_class_loss': 2.0/2, 'rpn_bbox_loss': 0.2/2, 'mrcnn_class_loss': 2.0, 'mrcnn_bbox_loss': 0.5/2, 'mrcnn_mask_loss': 1.0/2}
model.train(train_dataset, val_dataset, learning_rate=config.LEARNING_RATE, epochs=480, layers="2+")
model.config.LOSS_WEIGHTS = {'rpn_class_loss': 2.0/4, 'rpn_bbox_loss': 0.2/4, 'mrcnn_class_loss': 2.0, 'mrcnn_bbox_loss': 0.5/2, 'mrcnn_mask_loss': 1.0/2}
model.train(train_dataset, val_dataset, learning_rate=config.LEARNING_RATE, epochs=960, layers="all")
model.train(train_dataset, val_dataset, learning_rate=config.LEARNING_RATE/10, epochs=1020, layers="all")
model.train(train_dataset, val_dataset, learning_rate=config.LEARNING_RATE/100, epochs=1080, layers="all")


Starting at epoch 0. LR=0.001

Checkpoint Path: /gpfs01/bethge/home/iustyuzh/siamese-mask-rcnn/logs/ade20k20180612T1300/mask_rcnn_ade20k_{epoch:04d}.h5
Selecting layers to train
In model:  resnet_model
In model:  fpn_model
In model:  fpn_l1_model
In model:  rpn_model
Epoch 1/120
skip target
skip target
 974/1000 [============================>.] - ETA: 12s - loss: 1.6272 - rpn_class_loss: 0.2179 - rpn_bbox_loss: 0.1796 - mrcnn_class_loss: 0.4815 - mrcnn_bbox_loss: 0.2437 - mrcnn_mask_loss: 0.5045

Process Process-166:
Process Process-169:
Process Process-153:
Process Process-163:
Process Process-173:
Process Process-164:
Process Process-168:
Process Process-156:
Process Process-171:
Process Process-155:
Process Process-167:
Traceback (most recent call last):
Process Process-165:
Process Process-157:
Process Process-158:
Traceback (most recent call last):
Process Process-175:
Process Process-154:
Process Process-152:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Process Process-160:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/p

  File "<string>", line 2, in put
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 678, in _data_generator_task
    self.queue.put((True, generator_output))
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/managers.py", line 757, in _callmethod
    kind, result = conn.recv()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "<string>", line 2, in put
  File "/usr/lib/python3.6/multiprocessing/connection.py", lin

KeyboardInterrupt: 

In [ ]:
train_dataset.image_info[11]